In [24]:
import networkx as nx
from tqdm import tqdm
from datetime import datetime
import numpy as np
import random

In [25]:
file = open('ca-AstroPh.txt','r')
lines = file.readlines()

In [26]:
def dic2list(myDic):
    graph = []
    for i in myDic.keys():
        graph.append(i)
    return graph

In [27]:
nodes = {}
unique_nodes = set()
neighbor_c = {}
for l in lines:
    if '#' in str(l):
        continue
    node = l.replace('\n','').split('\t')
    n1 = int(node[0])
    n2 = int(node[1])
    nodes[(n1,n2)] = 1
    unique_nodes.add(n1)
    unique_nodes.add(n2)
    if  neighbor_c.get(n1,0) == 0:
             neighbor_c[n1] = []
    neighbor_c[n1].append(n2)
nodes_set = dic2list(nodes)

In [28]:
def reward(visited, queue, graph, node,out):
    visited.append(node)
    queue.append(node)
    i = 1
    while queue:
        s = queue.pop(0)
        if s == out:
            break
        i+=1
        # for neighbour in graph[s]:
        if graph.get(s,0) != 0:
           for neighbour in graph.get(s,0):
                if neighbour not in visited:
                    visited.append(neighbour)
                    queue.append(neighbour)
    return i

In [29]:
def iter_bfs(neigh,neighbor_probab):
    visited_n = []
    queue_n = []
    n_neighbor = {}
    for a in tqdm(neigh):
        n_neighbor[a] = reward(queue_n, visited_n,neighbor_probab,a)
    return n_neighbor

In [30]:
# iter_bfs(unique_nodes,neighbor_c)

In [31]:
def generate_realization_graph(nodes,p):
    neighbor_probable = {}
    nodes_probable_unique = set()
    nodes_probable =  nodes.copy()
    for i in list(nodes_probable.keys()):
        if np.random.uniform(0,1,1)[0] > p:
            del nodes_probable[i]
        else:
            nodes_probable_unique.add(i[0])
            nodes_probable_unique.add(i[1])
            if  neighbor_probable.get(i[0],0) == 0:
                 neighbor_probable[i[0]] = []
            if  neighbor_probable.get(i[1],0) == 0:
                 neighbor_probable[i[1]] = []
            neighbor_probable[i[0]].append(i[1])

    return nodes_probable,nodes_probable_unique,neighbor_probable

# nod = {(5,3):1,(5,7):1,(3,2):1,(3,4):1,(7,8):1,(4,8):1}
# x = generate_realization_graph(nod,1)[2]

In [32]:
# nod = {(5,3):1,(5,7):1,(3,2):1,(3,4):1,(7,8):1,(4,8):1}
# x = generate_realization_graph(nod,1)[2]

In [33]:

# my_bfs([],[],x,5,8)[1]

In [34]:
nodes_p,nodes_p_unique,neighbor_p =  generate_realization_graph(nodes,0.015)
len(nodes_p)

5949

In [35]:
r_nodes_graph = list(nodes_p_unique)
random.shuffle(r_nodes_graph)
outbreak = r_nodes_graph[0:10]
# print(type(outbreak))

In [36]:
rewards = {}
def calculate_node_reward( ):
    for n in tqdm(nodes_p_unique):
        n_list = []
        for o in outbreak:
             n_list.append(reward([],[],neighbor_p,o,n))
        rewards[n] = np.sum(n_list)

calculate_node_reward( )
print(rewards)


100%|██████████| 6419/6419 [01:41<00:00, 63.10it/s]


{3: 1875, 65546: 1875, 98315: 1875, 98318: 1875, 131091: 1875, 32791: 1875, 65564: 1875, 30: 1609, 65569: 1875, 98345: 1875, 43: 1875, 131121: 1875, 65595: 1875, 32840: 1875, 65613: 1875, 84: 1875, 88: 1875, 65629: 1875, 131167: 1770, 95: 1875, 65638: 1875, 65642: 1875, 131180: 1875, 98412: 1875, 114: 1875, 117: 1875, 120: 1875, 65657: 1875, 98432: 1875, 131204: 1408, 98439: 1875, 131209: 1875, 131221: 1875, 98456: 1875, 155: 1875, 131234: 1875, 164: 1875, 98471: 1875, 65713: 350, 178: 1875, 65723: 1875, 65724: 1875, 189: 1875, 131263: 274, 131266: 1875, 131272: 531, 131279: 1875, 98511: 1875, 65751: 1875, 131292: 1875, 98529: 1875, 229: 1875, 98541: 1875, 131315: 210, 131329: 1875, 65794: 1875, 65799: 1875, 65801: 1875, 98571: 1875, 131342: 1875, 33040: 1187, 276: 1875, 278: 1875, 65814: 1875, 65817: 1875, 98586: 1875, 33053: 1875, 98590: 1875, 65822: 1875, 289: 1875, 131361: 1875, 295: 1875, 33068: 1875, 131385: 1827, 131391: 1875, 319: 1875, 131396: 1875, 98629: 1875, 131400: 1875, 

In [37]:
sorted(rewards, key=lambda k: rewards[k], reverse=True)[0:10]

[3, 65546, 98315, 98318, 131091, 32791, 65564, 65569, 98345, 43]

### Greedy

In [38]:
def calculate_nodes_reward(nodes_p_unique,neighbor_p,outbreak ):
    reward_new = {}
    for n in tqdm(nodes_p_unique):
        n_list = []
        for o in outbreak:
             n_list.append(-1 * reward([],[],neighbor_p,o,n))
        reward_new[n] = np.sum(n_list)
    return reward_new



In [39]:
def hill_climing():
    S = []
    S.append(sorted(rewards, key=lambda k: rewards[k], reverse=True)[0])
    unique_nodes_h = nodes_p_unique.copy()
    outbreak_h = outbreak.copy()
    neighbor_p_c = neighbor_p.copy()
    outbreak.pop(0)
    neigh = set()
    for s in S:
        unique_nodes_h.remove(s)
        del neighbor_p_c[s]
    while len(S) < 4:
        temp_dic = {}
        temp_dic.update(calculate_nodes_reward(unique_nodes_h,neighbor_p_c,outbreak_h))
        n = sorted(temp_dic, key=lambda k: temp_dic[k], reverse=True)[0]
        S.append(n)
        unique_nodes_h.remove(n)
        del neighbor_p_c[n]
        outbreak.pop(0)
    return S
print(hill_climing())

100%|██████████| 6416/6416 [02:02<00:00, 52.50it/s]


[3, 38966, 89242, 45163]


In [40]:
# len(list(nx.bfs_tree(nx.DiGraph(dic2list(nodes_p)),3  )))

### Lazy Hill Climbing

In [41]:
mg_sorted = sorted(rewards, key=lambda k: rewards[k], reverse=True)
mg_sorted[0:10]

[3, 65546, 98315, 98318, 131091, 32791, 65564, 65569, 98345, 43]

In [42]:
def calculate_node_reward(node,neighbor_p,outbreak ):
    reward_new = {}

    n_list = []
    for o in outbreak:
         n_list.append(-1 * reward([],[],neighbor_p,o,node))
    reward_new[node] = np.sum(n_list)
    return reward_new

In [43]:
def lazy_hill_climbing():
    S = []
    mg_lh_sorted = mg_sorted.copy()
    S.append(mg_lh_sorted[0])
    unique_nodes_h = nodes_p_unique.copy()
    outbreak_h = outbreak.copy()
    neighbor_p_c = neighbor_p.copy()
    rewards_lhc = rewards.copy()
    outbreak_h.pop(0)
    for i , n in enumerate(mg_lh_sorted):
        if len (S) > 9:
            break
        left = n
        right = mg_sorted[i+1]
        l = rewards_lhc[left]
        r = rewards_lhc[right]
        if True :
            if i != 0:
                S.append(n)
            rewards[mg_lh_sorted[i+1]] = calculate_node_reward(mg_lh_sorted[i+1],neighbor_p_c,outbreak_h)
            unique_nodes_h.remove(n)
            del neighbor_p_c[n]
        else:
            neighbor_p_c = sorted(rewards, key=lambda k: rewards[k], reverse=True)
    return S


lazy_hill_climbing()


[3, 65546, 98315, 98318, 131091, 32791, 65564, 65569, 98345, 43]